In [1]:
!pip install transformers datasets accelerate sentencepiece --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling

In [2]:
!pip install wikipedia-api pandas unidecode

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.8 MB/s eta 0:00:00
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=6f591e04fdc147b89781610045727322d6ca9bd83c107b6e749d152ee4dced57
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [3]:
import os, random, re, gzip, requests
import wikipediaapi
import pandas as pd
import torch
from unidecode import unidecode
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [4]:
# ---- A) Wikipedia (temas variados) ----
wiki = wikipediaapi.Wikipedia(
    user_agent="ColabDatasetGenerator/1.0",
    language="es"
)

topics = [
    # Ciencia y tecnología
    "Inteligencia_artificial", "Aprendizaje_automático", "Red_neuronal",
    "Robótica", "Biotecnología", "Química_Orgánica", "Física_cuántica",
    "Teoría_de_la_relatividad", "Nanotecnología", "Ingeniería_genética",
    "Computadora", "Internet", "Ciberseguridad", "Criptografía",
    "Algoritmo", "Programación", "Sistemas_operativos", "Software_libre",
    "Base_de_datos", "Tecnología", "Electrónica", "Matemáticas",
    "Estadística", "Álgebra", "Cálculo", "Geometría",

    # Historia
    "Historia_de_Colombia", "Imperio_romano", "Antigua_Grecia",
    "Edad_Media", "Revolución_Francesa", "Primera_Guerra_Mundial",
    "Segunda_Guerra_Mundial", "Cristóbal_Colón", "Civilización_egipcia",
    "Civilización_maya", "Historiografía", "Arqueología",

    # Ciencia natural
    "Biología", "Zoología", "Botánica", "Genética", "Evolución",
    "Célula_(biología)", "Virus", "Bacteria", "Anatomía_humana",
    "Ecología", "Climatología", "Cambio_climático", "Geología",
    "Astronomía", "Sistema_solar", "Cosmología",

    # Medicina y salud
    "Medicina", "Farmacología", "Enfermedad", "Neurología",
    "Psiquiatría", "Psicología", "Nutrición", "Deporte", "Fisioterapia",

    # Humanidades
    "Filosofía", "Sociología", "Antropología", "Lingüística",
    "Lengua_española", "Gramática", "Literatura", "Poesía",
    "Narrativa", "Cultura", "Mitología", "Mitología_griega",
    "Mitología_romana", "Arte", "Arquitectura", "Música", "Cine",

    # Política y economía
    "Política", "Democracia", "Economía", "Microeconomía",
    "Macroeconomía", "Derecho", "Derechos_humanos", "Geopolítica",
    "Organización_de_las_Naciones_Unidas",

    # Geografía
    "Geografía", "América_del_Sur", "Europa", "Asia", "África",
    "Océano_Pacífico", "Montaña", "Río", "Clima",

    # Cultura popular
    "Videojuego", "Anime", "Ciencia_ficción", "Fantasía",
    "Red_social", "Internet_de_las_cosas",

    # Sociedad
    "Comunicación", "Periodismo", "Educación", "Aprendizaje",
    "Trabajo", "Economía_digital", "Globalización",

    # Otros temas útiles
    "Ética", "Lógica", "Creatividad", "Innovación", "Ingeniería",
    "Arquitectura_moderna", "Transporte", "Aviación",
    "Energía", "Energía_solar", "Energía_eólica",
    "Historia_de_la_ciencia",

    "Gabriel_García_Márquez", "Julio_Cortázar", "Pablo_Neruda",
    "Mario_Vargas_Llosa", "Benito_Pérez_Galdós", "Federico_García_Lorca",
    "Carlos_Fuentes", "Miguel_Ángel_Asturias",

    "Acentuación_del_español", "Ortografía_del_español", "Tilde_diacrítica",
    "Gramática_del_español", "Fonología_del_español", "Diptongo",
    "Hiato", "Acento_ortográfico"

    "Realismo_mágico", "El_Amor_en_los_Tiempos_del_Cólera", "Don_Quijote_de_la_Mancha",
    "Cien_años_de_soledad", "Literatura_latinoamericana", "Ñ"
]

import re

def clean_wiki_text(text):
    # Quitar secciones completas
    text = re.sub(r"==\s*Referencias\s*==.*", "", text, flags=re.DOTALL)
    text = re.sub(r"==\s*Bibliografía\s*==.*", "", text, flags=re.DOTALL)
    text = re.sub(r"==\s*Enlaces externos\s*==.*", "", text, flags=re.DOTALL)
    text = re.sub(r"==\s*Véase también\s*==.*", "", text, flags=re.DOTALL)
    text = re.sub(r"==\s*Notas\s*==.*", "", text, flags=re.DOTALL)

    # Quitar tablas y listas
    text = re.sub(r"\{.*?\}", "", text, flags=re.DOTALL)
    text = re.sub(r"^\*.*$", "", text, flags=re.MULTILINE)
    text = re.sub(r"\|\s*.*?\s*\|", "", text)

    return text.strip()


def download_wikipedia():
    txt = ""
    for t in topics:
        p = wiki.page(t)
        if p.exists() and len(p.text) > 2000:
            clean = clean_wiki_text(p.text)
            txt += clean + "\n"
    return txt

# ---- B) OpenSubtitles  ----
def download_opensubs():
    url = "https://object.pouta.csc.fi/OPUS-OpenSubtitles/v2018/mono/es.txt.gz"
    r = requests.get(url)
    path = "opensubs_es.gz"
    with open(path, "wb") as f:
        f.write(r.content)
    with gzip.open(path, "rt", encoding="utf-8", errors="ignore") as g:
        lines = []
        for i, line in enumerate(g):
            if 5 < len(line) < 180:
                lines.append(line.strip())
            if len(lines) >= 50000:  # límite para no explotar RAM
                break
        return "\n".join(lines)

print("Descargando Wikipedia...")
text_wiki = download_wikipedia()

print("Descargando OpenSubtitles...")
text_open = download_opensubs()

raw_text = "\n".join([text_wiki, text_open])
print("Caracteres totales obtenidos:", len(raw_text))

COMMON_MAP = {

    # ---- Verbo ser/estar ----
    "es": ["ez", "s", "es.", "es,"],
    "está": ["esta", "eta", "sta"],
    "esta": ["eta", "sta", "ta"],
    "están": ["estan", "etan"],
    "estan": ["estann", "etan", "etán"],
    "su": ["suu", "suu.", "suu,", "zu"],

    # ---- Funcionales frecuentes ----
    "muy": ["mui", "muuuy", "muuuyy", "mu"],
    "que": ["ke", "qe", "q", "k", "quee"],
    "para": ["pa", "pra", "para.", "x"],
    "pero": ["pro", "pero.", "peroo", "peor", "preo"],
    "porque": ["porqe", "por ke", "pk", "pq", "porq"],
    "también": ["tambien", "tmb", "tanbien"],
    "donde": ["dondee", "dnde"],
    "cuando": ["kndo", "cuanddo"],
    "solo": ["slo", "solo.", "sool"],
    "entonces": ["entoncs", "entnces"],

    # ---- Fonética b/v ----
    "bien": ["vien", "bn"],
    "vaso": ["baso"],
    "votar": ["botar"],
    "baca": ["vaca"],
    "voy": ["boi", "voi"],
    "veces": ["beces", "veses"],

    # ---- c/s/z ----
    "hacer": ["hazer", "acer"],
    "casa": ["caza", "cassa"],
    "cazar": ["casar", "cazar."],
    "pez": ["pes", "pez."],
    "azul": ["asul"],
    "cena": ["sena", "ceña"],

    # ---- g/j ----
    "gente": ["jente"],
    "girar": ["jirar"],
    "genial": ["jenial", "geniaal"],
    "jugar": ["gugar", "jugar."],

    # ---- ll/y ----
    "lluvia": ["yuvia", "lluvvia"],
    "llama": ["yama"],
    "calle": ["calye", "caye"],
    "lleno": ["yeno"],

    # ---- Omisión de h ----
    "hoy": ["oy"],
    "hasta": ["asta"],
    "había": ["avia"],
    "hace": ["ace"],
    "hablar": ["ablar"],

    # ---- Teclado ----
    "hola": ["hloa", "ola", "holaa", "holla"],
    "gracias": ["grasias", "grcias", "gracis"],
    "amigo": ["amgio", "amigo.", "amigoo"],
    "quiero": ["qiero", "kiero", "quieroo"],
    "cualquier": ["cauqlier", "cualkier"],

    # ---- Colloquiales ----
    "sí": ["si", "sii", "siii"],
    "no": ["n0", "noo", "nooo"],
    "estoy": ["toy", "estoi"],

    # ---- Confusiones semánticas ----
    "haya": ["halla", "aya"],
    "halla": ["haya", "aya"],
    "valla": ["vaya"],
    "vaya": ["valla"],
    "echo": ["hecho"],
    "hecho": ["echo"],
    "iba": ["iva"],

    # ---- Palabras con acento ----
    "mamá": ["mama", "mamaa"],
    "papá": ["papa", "papaa"],
    "corazón": ["corazon"],
    "camión": ["camion"],
    "adiós": ["adios"],

    # ---- Duplicación ----
    "viene": ["vienee"],
    "todo": ["todoo"],

    # ---- Otros comunes ----
    "sabe": ["zabe", "save", "sabe."],
    "fea": ["flea", "fia"],
    "bonita": ["vonita", "bonitta"],
    "pequeña": ["pekeña", "pequena"],
    "tener": ["tenerr", "tener.", "tenr"],
    "decir": ["desir"],
    "hacerlo": ["acerlo"],

    # ---- Acentos esenciales ----
    "él": ["el"],
    "más": ["mas"],
    "sí": ["si"],
    "mí": ["mi"],
    "tú": ["tu"],
    "sólo": ["solo"],
    "aún": ["aun"],
    "cómo": ["como"],
    "cuándo": ["cuando"],
    "dónde": ["donde"],
    "qué": ["que"],
    "quién": ["quien"],
    "cuánto": ["cuanto"],
    "cuántos": ["cuantos"],
    "cuánta": ["cuanta"],
    "cuántas": ["cuantas"],
    "porqué": ["porque"],
    "por qué": ["por que"],

    # ---- Sustantivos acentuados comunes ----
    "mamá": ["mama"],
    "papá": ["papa"],
    "camión": ["camion"],
    "televisión": ["television"],
    "acción": ["accion"],
    "lógica": ["logica"],
    "música": ["musica"],
}

#-----------------------
sentences = re.findall(r'[^.!?]+[.!?]', raw_text)
paragraphs = raw_text.split("\n")

clean_text_units = []

for s in sentences:
    s = s.strip()
    if 5 < len(s.split()) < 40:
        clean_text_units.append(s)

for p in paragraphs:
    p = p.strip()
    if 25 < len(p.split()) < 120:
        clean_text_units.append(p)

random.shuffle(clean_text_units)
clean_text_units = clean_text_units[:70000]

#    4. GENERADOR DE ERRORES
PHONETIC = {
    "b":"v","v":"b",
    "c":"s","s":"c","z":"s",
    "ll":"y","y":"ll",
    "qu":"k","k":"qu",
    "g":"j","j":"g",
    "h":""
}

ACCENTS = {
    "a":"á","á":"a","e":"é","é":"e",
    "i":"í","í":"i","o":"ó","ó":"o",
    "u":"ú","ú":"u",
    "n":"ñ", "ñ":"n"
}

HOMOPHONES = [
    ("casa","caza"), ("tubo","tuvo"), ("echo","hecho"),
    ("vaya","valla"), ("haya","halla"), ("haber","a ver"),
    ("abría","habría"),
    ("ciento","siento"),
    ("honda","onda")
]

def corrupt_word(w):
    original = w
    base = w.lower()

    if base in COMMON_MAP and random.random() < 0.70:
        base = random.choice(COMMON_MAP[base])
        if original.istitle(): base = base.capitalize()
        if original.isupper(): base = base.upper()
        return base

    # 1. homófonos
    for a,b in HOMOPHONES:
        if base == a and random.random() < 0.3: return b
        if base == b and random.random() < 0.3: return a

    # 2. quitar acentos
    if random.random() < 0.50:
        base = unidecode(base)

    # 3. fonética
    for k,v in PHONETIC.items():
        if k in base and random.random() < 0.2:
            base = base.replace(k, v)

    # >>> AÑADIDO: insertar "h" aleatoriamente
    if random.random() < 0.1:
        pos = random.randint(0, len(base))
        base = base[:pos] + "h" + base[pos:]

    # >>> AÑADIDO: unir palabras pequeñas (ej: "de el" → "del")
    if random.random() < 0.08:
        if base in ["de","a","por","para","con"]:
            base = base + random.choice(["l","la","lo","el"])

    # 4. letras básicas
    if len(base) > 3 and random.random() < 0.4:
        r = random.random()
        i = random.randint(0, len(base)-1)
        if r < 0.25:
            base = base[:i] + base[i]*2 + base[i+1:]
        elif r < 0.5:
            base = base[:i] + base[i+1:]
        elif r < 0.75:
            base = base[:i] + random.choice("abcdefghijklmnopqrstuvwxyz") + base[i+1:]
        else:
            if i < len(base)-1:
                lst = list(base)
                lst[i], lst[i+1] = lst[i+1], lst[i]
                base = "".join(lst)

    # 5. acentos incorrectos
    if random.random() < 0.3:
        chars = list(base)
        for j,ch in enumerate(chars):
            if ch in ACCENTS and random.random() < 0.2:
                chars[j] = ACCENTS[ch]
        base = "".join(chars)

    if original.istitle(): base = base.capitalize()
    if original.isupper(): base = base.upper()
    return base

    # Añadir tildes incorrectas (ej. "dónde" → "dóndé")
    if random.random() < 0.15:
      accents = {"a":"á","e":"é","i":"í","o":"ó","u":"ú"}
      pos = random.randint(0, len(base)-1)
      if base[pos] in accents:
        base = base[:pos] + accents[base[pos]] + base[pos+1:]

def corrupt_sentence(text):
    words = text.split()
    out = []
    for w in words:
        if random.random() < 0.5:
            out.append(corrupt_word(w))
        else:
            out.append(w)

    # reordenamiento contextual
    if len(out) > 6 and random.random() < 0.25:
        i = random.randint(0, len(out)-3)
        j = i + random.randint(1, 2)
        out[i], out[j] = out[j], out[i]

    return " ".join(out)

pairs = []
TARGET_SIZE = 200_000

for t in clean_text_units:
    for _ in range(3):
        corrupted = corrupt_sentence(t)
        pairs.append({"input_text": corrupted, "target_text": t})
        if len(pairs) >= TARGET_SIZE:
            break
    if len(pairs) >= TARGET_SIZE:
        break

df = pd.DataFrame(pairs)
df.to_csv("dataset_errores_wikipedia.csv", index=False, encoding="utf-8")

print("Dataset generado:", len(df), "ejemplos")
print(df.sample(5))

Descargando Wikipedia...
Descargando OpenSubtitles...
Caracteres totales obtenidos: 7958692
Dataset generado: 175128 ejemplos
                                               input_text  \
84375   En 193, el rey Leopoldo III de Belgica proclám...   
15333   Sin embargo, el registro no es baundanté en fo...   
62677   Eñ opinión de algunos ecpecialictacx en ta fsa...   
144935         Si kicre que repasemos todo hota vez, iré.   
169737               Prometiste daihme 5 rllos por ellos.   

                                              target_text  
84375   En 1936, el rey Leopoldo III de Bélgica procla...  
15333   Sin embargo, el registro no es abundante en fo...  
62677   En opinión de algunos especialistas, en esta f...  
144935        Si quiere que repasemos todo otra vez, iré.  
169737                 Prometiste darme 5 ryos por ellos.  


In [5]:
# --- Cargar modelo y tokenizer ---
MODEL = "vgaraujov/bart-base-spanish"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL).to("cuda")

# CONGELAR CAPAS

for param in model.model.encoder.parameters():
    param.requires_grad = False

num_layers = len(model.model.decoder.layers)
layers_to_train = 2

for i, layer in enumerate(model.model.decoder.layers):
    if i < num_layers - layers_to_train:
        for param in layer.parameters():
            param.requires_grad = False
    else:
        print(f"Entrenando DECODER capa {i}")

print("Capas congeladas y listas para entrenamiento.")

# --- Cargar dataset generado ---
dataset = load_dataset("csv", data_files="dataset_errores_wikipedia.csv")["train"]
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_ds, val_ds = dataset["train"], dataset["test"]

# --- Preprocesamiento ---
def preprocess(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=64, truncation=True)
    labels = tokenizer(examples["target_text"], max_length=64, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
tokenized_val = val_ds.map(preprocess, batched=True, remove_columns=val_ds.column_names)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Entrenando DECODER capa 4
Entrenando DECODER capa 5
Capas congeladas y listas para entrenamiento.


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/157615 [00:00<?, ? examples/s]

Map:   0%|          | 0/17513 [00:00<?, ? examples/s]

In [6]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./bart-correccion",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    num_train_epochs=4,
    fp16=True,
    eval_strategy="epoch",
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=3,
    report_to="none"
)

In [7]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.766500,0.581606
2,0.669000,0.510618
3,0.606000,0.480798
4,0.578500,0.471860


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=39404, training_loss=0.6922768219729122, metrics={'train_runtime': 2689.5596, 'train_samples_per_second': 234.41, 'train_steps_per_second': 14.651, 'total_flos': 2.376032463028224e+16, 'train_loss': 0.6922768219729122, 'epoch': 4.0})

In [14]:
text = "vamoz a ugar a la play."
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, num_beams=4, max_length=64)
print("\nTexto original:", text)
print("Corregido:", tokenizer.decode(outputs[0], skip_special_tokens=True))


Texto original: vamoz a ugar a la play.
Corregido: Vamos a jugar a la play.
